In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from collections import defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
sns.set_theme(style="ticks")

#https://github.com/qpwo/pySCF
#from social_choice.profile import Profile#, ballot_box, plurality

#https://github.com/djcunningham0/rankaggregation
import rankaggregation as ra

# Read Submission

In [2]:
base_path      = '/media/workspace/booking_challenge/output/evaluation/EvaluationTask/results/'

# Local

model1  = "SupervisedModelTraining____mars_gym_model_b____ae4b748000_ecd98441ed"
model2 = "SupervisedModelTraining____mars_gym_model_b____8ee4956f7b_58ecb72721"
model3 = "SupervisedModelTraining____mars_gym_model_b____7b80cf80f0_8ee798d706"
model4 = "SupervisedModelTraining____mars_gym_model_b____4c23d3f48d_941b2a9df2"


files          = [model1, model2, model3, model4]
model_names    = ['model1', 'model2', 'model3','model4']

In [3]:

df_sub = {}
for i in range(len(files)):
    path = base_path+files[i]+'/all_reclist_{}.csv'.format(files[i].split("/")[-1])
    _df   = pd.read_csv(path)
    _df['reclist'] = _df['reclist'].apply(eval)
    df_sub[model_names[i]] = _df

df_sub.keys()

dict_keys(['model1', 'model2', 'model3', 'model4'])

In [4]:
df_sub['model1'].head()

,utrip_id,reclist
0,1000033_1,"[38677, 48968, 49087, 34123, 8335, 12320, 2919..."
1,1000045_1,"[36063, 17127, 60002, 58178, 2122, 23940, 699,..."
2,1000212_1,"[18417, 23714, 29738, 3505, 66648, 7240, 47389..."
3,1000241_1,"[53363, 56198, 30688, 18508, 32815, 12430, 174..."
4,1000876_1,"[17013, 63151, 4932, 60222, 15343, 11652, 5343..."


In [5]:
df_sub.keys()

dict_keys(['model1', 'model2', 'model3', 'model4'])

## Ranked

In [6]:
# import itertools
# import numpy as np
# from collections import OrderedDict

# # list(df_sub.keys())
# def get_list(df_sub, uid, models = []):
#     return list(OrderedDict.fromkeys(list(itertools.chain(*[list(df_sub[k].iloc[uid].values) 
#                                                  for k in models]))))

# candidates = get_list(df_sub, 0, ['model', 'coocorrence', 'most_popular'])
# candidates

In [7]:
def get_rerank(reclist, type = 'instant_runoff'):
    if type == 'instant_runoff':
        return agg.instant_runoff(ballots)
    elif type == 'borda':
        l = agg.borda(ballots)
    elif type == 'dowdall':
        l = agg.dowdall(ballots)
    elif type == 'average_rank':
        l = agg.average_rank(ballots)
    
    return [i[0] for i in l]

agg = ra.RankAggregator()

In [11]:
#df_sub['model1']

In [16]:
methods = ['dowdall', 'borda',  'average_rank', 'instant_runoff']
reclist_size = 4

for kind_rerank in methods:
    print(kind_rerank, "...")
    df_final = df_sub['model1'].copy()
    
    reranked_list = []
    for uid in tqdm(range(len(df_sub['model1']))):
        
        ballots  = [list(df_sub[k].iloc[uid]['reclist']) for k in model_names]
        reranked = list(np.array(get_rerank(ballots, kind_rerank)).astype(int))[:reclist_size]
        reranked_list.append(reranked)
    
    #np.savetxt('metadada/final_submission_{}.csv'.format(kind_rerank), reranked_list, fmt='%i', delimiter=',') 

    #def save_submission(self, df_metric, rank_list):

    df_final['reclist']  = list(reranked_list)
    df_final['city_id_1'] = df_final['reclist'].apply(lambda reclist: reclist[0])
    df_final['city_id_2'] = df_final['reclist'].apply(lambda reclist: reclist[1])
    df_final['city_id_3'] = df_final['reclist'].apply(lambda reclist: reclist[2])
    df_final['city_id_4'] = df_final['reclist'].apply(lambda reclist: reclist[3])

    # base submission
    df_final[['utrip_id', 'city_id_1', 'city_id_2', 'city_id_3', 'city_id_4']]\
        .to_csv('metadada/final_submission_{}.csv'.format(kind_rerank), index=False)    

  1%|▏         | 290/21671 [00:00<00:15, 1412.38it/s]

dowdall ...


  0%|          | 15/21671 [00:00<02:27, 146.39it/s]

borda ...


  0%|          | 13/21671 [00:00<03:03, 118.14it/s]

average_rank ...


  0%|          | 0/21671 [00:00<?, ?it/s]

instant_runoff ...


ValueError: candidates must be a list or string